<a href="https://colab.research.google.com/github/luisosmx/web-scraperf1/blob/main/f1_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime


# GET request to URL:
url = "https://www.formula1.com/en/results.html/2022/fastest-laps.html"
response = requests.get(url)

# Analizar el contenido HTML usando BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Encontrar la tabla de resultados para los conductores
table = soup.find("table", attrs={"class": "resultsarchive-table"})

# Extraer los datos de la tabla en un DataFrame de pandas
df = pd.read_html(str(table))[0]

# Seleccionar sólo las columnas necesarias
df = df[["Grand Prix", "Driver", "Car", "Time"]]

# Crear una nueva columna "Acronym" con los últimos 3 caracteres de la columna "Driver"
df["Acronym"] = df["Driver"].str.slice(-3)

# Quitar los últimos 3 caracteres de la columna "Driver"
df["Driver"] = df["Driver"].str.slice(stop=-3)


now = datetime.now()

epoch_time = int(now.timestamp())
  
df["Date"] = epoch_time

new_name = {"Car": "Team"}
df = df.rename(columns=new_name)

# Convertir la columna "PTS" a un tipo de datos de cadena
#df["Laps"] = df["Laps"].astype(str)
df["Date"] = df["Date"].astype(str)

# Descarga del df en archivo .CSV
df.to_csv("resultados_teams.csv", index=False)

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar -xzf spark-3.1.2-bin-hadoop3.2.tgz
!mv spark-3.1.2-bin-hadoop3.2.tgz /opt/spark-3.1.2
!pip install -q findspark
!pip install pyspark==3.1.2 
!ln -s /opt/spark-3.1.2 /opt/spark
!export SPARK_HOME=/opt/spark
!export PATH=$SPARK_HOME/bin:$PATH

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 19.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880770 sha256=9ade83775ddbd6216e717fef3565d0c130de40ac0f3eb9c8b0607a8f433facd7
  Stored in directory: /root/.cache/pip/wheels/11/17/0b/53e7d10fe66ca7647d391cdba323fcf5b2f9dfcb7ebad87aa7
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
!pip install pyspark==3.1.2 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import findspark

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.1.2-bin-hadoop3.2'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk:1.10.34,org.apache.hadoop:hadoop-aws:2.6.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk:1.10.34,org.apache.hadoop:hadoop-aws:2.7.2 pyspark-shell'

findspark.init()

sc = pyspark.SparkContext("local[*]")
spark = SparkSession(sc)

print('Modules imported and Spark loaded')


Modules imported and Spark loaded


In [9]:
!wget --continue /content/resultados_teams.csv

/content/resultados_teams.csv: Scheme missing.


In [11]:
spark_df = spark.read.csv('/content/resultados_teams.csv')
spark_df.show()

+--------------+----------------+--------------------+--------+-------+----------+
|           _c0|             _c1|                 _c2|     _c3|    _c4|       _c5|
+--------------+----------------+--------------------+--------+-------+----------+
|    Grand Prix|          Driver|                Team|    Time|Acronym|      Date|
|       Bahrain|Charles Leclerc |             Ferrari|1:34.570|    LEC|1681183865|
|  Saudi Arabia|Charles Leclerc |             Ferrari|1:31.634|    LEC|1681183865|
|     Australia|Charles Leclerc |             Ferrari|1:20.260|    LEC|1681183865|
|Emilia Romagna| Max Verstappen |Red Bull Racing RBPT|1:18.446|    VER|1681183865|
|         Miami| Max Verstappen |Red Bull Racing RBPT|1:31.361|    VER|1681183865|
|         Spain|   Sergio Perez |Red Bull Racing RBPT|1:24.108|    PER|1681183865|
|        Monaco|   Lando Norris |    McLaren Mercedes|1:14.693|    NOR|1681183865|
|    Azerbaijan|   Sergio Perez |Red Bull Racing RBPT|1:46.046|    PER|1681183865|
|   

In [12]:
df = spark.read.option("header",True).csv('/content/resultados_teams.csv')
df.show()

+--------------+----------------+--------------------+--------+-------+----------+
|    Grand Prix|          Driver|                Team|    Time|Acronym|      Date|
+--------------+----------------+--------------------+--------+-------+----------+
|       Bahrain|Charles Leclerc |             Ferrari|1:34.570|    LEC|1681183865|
|  Saudi Arabia|Charles Leclerc |             Ferrari|1:31.634|    LEC|1681183865|
|     Australia|Charles Leclerc |             Ferrari|1:20.260|    LEC|1681183865|
|Emilia Romagna| Max Verstappen |Red Bull Racing RBPT|1:18.446|    VER|1681183865|
|         Miami| Max Verstappen |Red Bull Racing RBPT|1:31.361|    VER|1681183865|
|         Spain|   Sergio Perez |Red Bull Racing RBPT|1:24.108|    PER|1681183865|
|        Monaco|   Lando Norris |    McLaren Mercedes|1:14.693|    NOR|1681183865|
|    Azerbaijan|   Sergio Perez |Red Bull Racing RBPT|1:46.046|    PER|1681183865|
|        Canada|   Carlos Sainz |             Ferrari|1:15.749|    SAI|1681183865|
| Gr

In [13]:
spark.read.option("header",True) \
          .csv("/content/resultados_teams.csv") \
          .createOrReplaceTempView("drivers_f1_file")

In [15]:
spark.sql("SELECT * FROM drivers_f1_file").show()

+--------------+----------------+--------------------+--------+-------+----------+
|    Grand Prix|          Driver|                Team|    Time|Acronym|      Date|
+--------------+----------------+--------------------+--------+-------+----------+
|       Bahrain|Charles Leclerc |             Ferrari|1:34.570|    LEC|1681183865|
|  Saudi Arabia|Charles Leclerc |             Ferrari|1:31.634|    LEC|1681183865|
|     Australia|Charles Leclerc |             Ferrari|1:20.260|    LEC|1681183865|
|Emilia Romagna| Max Verstappen |Red Bull Racing RBPT|1:18.446|    VER|1681183865|
|         Miami| Max Verstappen |Red Bull Racing RBPT|1:31.361|    VER|1681183865|
|         Spain|   Sergio Perez |Red Bull Racing RBPT|1:24.108|    PER|1681183865|
|        Monaco|   Lando Norris |    McLaren Mercedes|1:14.693|    NOR|1681183865|
|    Azerbaijan|   Sergio Perez |Red Bull Racing RBPT|1:46.046|    PER|1681183865|
|        Canada|   Carlos Sainz |             Ferrari|1:15.749|    SAI|1681183865|
| Gr